Because the datasets are SO large (especially the Multiome dataset), instead of running both parts of the project in one notebook (and risk Kaggle running out of storage space then resetting all progress), it is more convenient to separate the multiome and citeseq parts of the project, then later merge the predicted outputs from the two parts together.

This notebook concerns itself with the CITEseq portion.

# First, all the basic imports and file names which may or may not be used is loaded in essentially as a header

In [ ]:
! pip install tables

In [ ]:
import os, gc, pickle, datetime, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display

import scipy.sparse

In [ ]:
# Directory of the data
DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULT_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULT_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULT_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_MULT_TRAIN_TARGETS_idx = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz"
FP_MULT_TRAIN_TARGETS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz"
FP_MULT_TRAIN_INPUTS_idx = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_idxcol.npz"
FP_MULT_TRAIN_INPUTS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_values.sparse.npz"
FP_MULT_TEST_INPUTS_idx = "../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz"
FP_MULT_TEST_INPUTS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_values.sparse.npz"

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

FP_EVALUATION_IDS_parquet = "../input/multimodal-single-cell-as-sparse-matrix/evaluation.parquet"

multi_ome_only_file = '../input/n256-nb2-multiome/multiome_only_256.csv'

# CITEseq Part: Predicting protein levels

Now the CITEseq portion begins


Code from pourchot: https://www.kaggle.com/code/pourchot/all-in-one-citeseq-multiome-with-keras

In [ ]:
svd_ncount = 256 # amount of dimensions to keep for SVD later

## Load in the data

In [ ]:
# Load training data
X = pd.read_hdf(FP_CITE_TRAIN_INPUTS)
Y = pd.read_hdf(FP_CITE_TRAIN_TARGETS)

# Load test inputs
X_test = pd.read_hdf(FP_CITE_TEST_INPUTS)

Constant columns (a.k.a. columns that have the same value in all rows) are useless for machine learning. Just like if you are told to differentiate between apples and oranges, and there is a column which indicates whether apples and oranges are fruits and vegetables, both the apples and oranges will be "fruit," which informs you nothing about the difference between apples and oranges.

Hence, constant columns found in the training inputs are found in order to be removed from the input data.

In [ ]:
constant_cols = list(X.columns[(X == 0).all(axis=0).values]) +\
                list(X_test.columns[(X_test == 0).all(axis=0).values])
print('constant columns ',len(constant_cols))

In [ ]:
# remove the constant columns from the training data
X = X.drop(columns = constant_cols)
Xt = X_test.drop(columns = constant_cols)

The "important columns" are columns that appear as training targets. Hence, it is considered important to keep them in mind

In [ ]:
important_cols = []
for y_col in Y.columns:
    important_cols += [x_col for x_col in X.columns if y_col in x_col]
print('important columns ',len(important_cols))

Before this point, the training and testing data has been loaded in order to determine the constant columns. The training and testing data will be loaded now as sparse matrices with the constant columns removed and the important columns kept. The purpose of sparse matrices is to efficiently store data with lots of zeros and also speed up the machine learning processes.

In [ ]:
# First, taking a look at X shows there are a LOT of zeros:
X.head()

In [ ]:
# first delete the X, X_test, Xt, and Y to save space
del X
del X_test
del Xt
del Y

# load in the metadata since it'll be modified as well in the next cell
# (Since X and Y are modified, it is convenient to modify the metadata to match
# at the same time)
metadata_df = pd.read_csv(FP_CELL_METADATA, index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"] # focus on citeseq right now
metadata_df.shape # show the shape

In [ ]:
%%time
# 2min 17s

# Now, the data will be converted into sparse matrices
# (See MSCI CITEseq Keras Quickstart by AMBROSM)

# Read train and convert to sparse matrix
X = pd.read_hdf(FP_CITE_TRAIN_INPUTS).drop(columns=constant_cols)
cell_index = X.index
meta = metadata_df.reindex(cell_index)
X0 = X[important_cols].values
print(f"Original X shape: {str(X.shape):14} {X.size*4/1024/1024/1024:2.3f} GByte")
gc.collect()
X = scipy.sparse.csr_matrix(X.values)
gc.collect()

# Read test and convert to sparse matrix
Xt = pd.read_hdf(FP_CITE_TEST_INPUTS).drop(columns=constant_cols)
cell_index_test = Xt.index
meta_test = metadata_df.reindex(cell_index_test)
X0t = Xt[important_cols].values
print(f"Original Xt shape: {str(Xt.shape):14} {Xt.size*4/1024/1024/1024:2.3f} GByte")
gc.collect()
Xt = scipy.sparse.csr_matrix(Xt.values)

## Perform SVD
Now perform SVD in order to reduce the number of features

In [ ]:
%%time
# 5-6 minutes

# Apply the singular value decomposition
both = scipy.sparse.vstack([X, Xt])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=svd_ncount, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X = both[:70988]
Xt = both[70988:]
del both
X = np.hstack([X, X0])
Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X.shape):14} {X.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(Xt.shape):14} {Xt.size*4/1024/1024/1024:2.3f} GByte")

In [ ]:
print("Explained variance:")
print(svd.explained_variance_ratio_.sum())

In [ ]:
# Read Y
Y = pd.read_hdf(FP_CITE_TRAIN_TARGETS)
y_columns = list(Y.columns)
Y = Y.values

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
Y -= Y.mean(axis=1).reshape(-1, 1)
Y /= Y.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(Y.shape):14} {Y.size*4/1024/1024/1024:2.3f} GByte")

## CITEseq learning model

From: https://www.kaggle.com/code/pourchot/all-in-one-citeseq-multiome-with-keras/notebook

In [ ]:
import math

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, BatchNormalization

metric and loss function from MSCI CITEseq Keras Quickstart by AMBROSM



In [ ]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

def negative_correlation_loss(y_true, y_pred):
    """Negative correlation loss function for Keras
    
    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1
    
    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """
    my = K.mean(tf.convert_to_tensor(y_pred), axis=1)
    my = tf.tile(tf.expand_dims(my, axis=1), (1, y_true.shape[1]))
    ym = y_pred - my
    r_num = K.sum(tf.multiply(y_true, ym), axis=1)
    r_den = tf.sqrt(K.sum(K.square(ym), axis=1) * float(y_true.shape[-1]))
    r = tf.reduce_mean(r_num / r_den)
    return - r

In [ ]:
LR_START = 0.01
BATCH_SIZE = 512

def create_model():
    
    reg1 = 9.613e-06
    reg2 = 1e-07
    REG1 = tf.keras.regularizers.l2(reg1)
    REG2 = tf.keras.regularizers.l2(reg2)
    DROP = 0.1

    activation = 'selu'
    inputs = Input(shape =(X.shape[1],))

    x0 = Dense(256, 
              kernel_regularizer = REG1,
              activation = activation,
             )(inputs)
    x0 = Dropout(DROP)(x0)
    
    
    x1 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x0)
    x1 = Dropout(DROP)(x1)
    
    
    x2 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x1) 
    x2= Dropout(DROP)(x2)
    
    x3 = Dense(Y.shape[1],
               kernel_regularizer = REG1,
               activation = activation,
             )(x2)
    x3 = Dropout(DROP)(x3)

         
    x = Concatenate()([
                x0, 
                x1, 
                x2, 
                x3
                ])
    
    x = Dense(Y.shape[1], 
                kernel_regularizer = REG2,
                activation='linear',
                )(x)
    
    
    model = Model(inputs, x)
    

    return model

In [ ]:
%%time
# 13 min 44 s
VERBOSE = 1

import warnings
warnings.filterwarnings("ignore")

EPOCHS = 50 
N_SPLITS = 3

pred_train = np.zeros((Y.shape[0],Y.shape[1]))

np.random.seed(1)
tf.random.set_seed(1)
score_list = []
kf = GroupKFold(n_splits=N_SPLITS)
score_list = []

for fold, (idx_tr, idx_va) in enumerate(kf.split(X, groups=meta.donor)):
    start_time = datetime.datetime.now()
    model = None
    gc.collect()
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    X_va = X[idx_va]
    y_va = Y[idx_va]

    lr = ReduceLROnPlateau(
                    monitor = "val_loss",
                    factor = 0.9, 
                    patience = 4, 
                    verbose = VERBOSE)

    es = EarlyStopping(
                    monitor = "val_loss",
                    patience = 40, 
                    verbose = VERBOSE,
                    mode = "min", 
                    restore_best_weights = True)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                    filepath = './citeseq',
                    save_weights_only = True,
                    monitor = 'val_loss',
                    mode = 'min',
                    save_best_only = True)

    callbacks = [
                    lr, 
                    es, 
                    model_checkpoint_callback
                    ]
    
    model = create_model()
    
    model.compile(
                optimizer = tf.keras.optimizers.Adam(learning_rate=LR_START),
                metrics = [negative_correlation_loss],
                loss = negative_correlation_loss
                 )
    # Training
    model.fit(
                X_tr,
                y_tr, 
                validation_data=(
                                X_va,
                                y_va), 
                epochs = EPOCHS,
                verbose = VERBOSE,
                batch_size = BATCH_SIZE,
                shuffle = True,
                callbacks = callbacks)

    del X_tr, y_tr 
    gc.collect()
    
    model.load_weights('./citeseq')
    model.save(f"./submissions/model_{fold}")
    print('model saved')
    
    #  Model validation
    y_va_pred = model.predict(X_va)
    corrscore = correlation_score(y_va, y_va_pred)
    pred_train[idx_va] = y_va_pred
    
    print(f"Fold {fold}, correlation =  {corrscore:.5f}")
    del X_va, y_va, y_va_pred
    gc.collect()
    score_list.append(corrscore)

# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Mean corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
score_total = correlation_score(Y, pred_train)
print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

## Predictions for CITEseq

In [ ]:
%%time
# Around 20 s

test_pred = np.zeros((len(Xt), 140), dtype=np.float32)
for fold in range(N_SPLITS):
    print(f"Predicting with fold {fold}")
    model = load_model(f"./submissions/model_{fold}",
                       custom_objects={'negative_correlation_loss': negative_correlation_loss})
    test_pred += model.predict(Xt)

## Save submission by merging with multiome

In [ ]:
%%time
# 2min 41s

# Merge with multiome
submission = pd.read_csv(multi_ome_only_file,index_col='row_id', squeeze=True)
submission.iloc[:len(test_pred.ravel())] = test_pred.ravel()
assert not submission.isna().any()

submission.to_csv('submission_full_m256_c256.csv')
display(submission)